In [2]:

!pip install opencv-python
!pip install matplotlib


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/a4/7d/f1c30a92854540bf789e9cd5dde7ef49bbe63f855b85a2e6b3db8135c591/opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/39.5 MB 3.4 MB/s eta 0:00:12
   ---------------------------------------- 0.1/39.5 MB 2.3 MB/s eta 0:00:18
   ---------------------------------------- 0.1/39.5 MB 751.6 kB/s eta 0:00:53
   ---------------------------------------- 0.1/39.5 MB 656.4 kB/s eta 0:01:00
   ---------------------------------------- 0.1/39.5 MB 656.4 kB/s eta 0:01:00
   ---------------------------------------- 0.1/39.5 MB 481.4 kB/s eta 0:01:22
   ---------------------------------------- 0.2/39.5 MB 546.6 kB/s eta 0:01:12
   ---------------------------------------- 0.2/39.5 MB 590.8 kB/s eta 0:01:07
   ---------------------------------------- 0.2/39.5 MB 590.8 kB/s eta 0:01:07
 

In [3]:
import cv2
import numpy as np


In [4]:
import os
print(os.path.exists("colorization_release_v2.caffemodel"))


True


In [5]:
print("Models loading...")
net= cv2.dnn.readNetFromCaffe('colorization_deploy_v2.prototxt', "colorization_release_v2.caffemodel")
pts= np.load('pts_in_hull.npy')


Models loading...


In [6]:
if 'net' in globals():
    class8 = net.getLayerId("class8_ab")
    conv8 = net.getLayerId("conv8_313_rh")
    pts = pts.transpose().reshape(2,313,1,1)
else:
    print("Error: Model 'net' is not loaded properly.")


In [7]:
class8 = net.getLayerId("class8_ab")
conv8 =  net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2,313,1,1)
                        

In [8]:
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs =  [np.full([1,313],2.606,dtype='float32')]

In [9]:
image = cv2.imread('girlimage.jpg')
scaled = image.astype("float32")/255.0
lab = cv2.cvtColor(scaled,cv2.COLOR_BGR2LAB)

In [10]:
resized = cv2.resize(lab,(224,224))
L = cv2.split(resized)[0]
L -= 50

In [11]:
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1,2,0))


In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Resize 'ab' channels to match the original image size
ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

# Extract the L channel from the LAB image
L = cv2.split(lab)[0]
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

# Convert from LAB to BGR color space
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)

# Convert to 8-bit image
colorized = (255 * colorized).astype("uint8")

# Resize the colorized image for better display (optional)
display_size = (600, 400)  # Adjust dimensions as needed
colorized_resized = cv2.resize(colorized, display_size)

# Display using OpenCV (normal way)
cv2.imshow("Original", cv2.resize(image, display_size))
cv2.imshow("Colorized", colorized_resized)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Alternative: Display using Matplotlib (if OpenCV causes size issues)
plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(colorized, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
plt.axis("off")  # Hide axes
plt.show()
